<h1>Capstone Project - The Battle of Neighborhoods (Week 4 and 5)</h1>

<h2>Introduction</h2> 
<p>For the final project, I would like to pizza restaurants in New York.</p>

<h2>Business Problem Section</h2> 
<h3>Background</h3>
<p>Luigi is the CEO of Luigi's Pizza Joint, which started as a family owned and operated business in the 60's. Luigi already has a number of pizza restaurants in other cities in America, but he would like to expand their gourmet pizza business in New York, specifically Manhattan.</p> 
<h3>Business Problem</h3>
<p>Luigi is concerned that there are already a number of pizza businesses in New York, specifically Manhattan, however none are to the same standard has Luigi's Pizza Joint. In this scenario, the aim is to identify a suitable location, on what basis is this choice suitable and identify the competition in that location.</p>
<p>The audience for the results will be Luigi and his five brothers who are also part directors of other Luigi's Pizza Joints across America.</p>

<h2>Data Section</h2> 
<h3>Data Sources</h3>
<p>Before choosing this topic I did try and locate data for Perth, Western Australia, however nothing free is available. Therefore I have chosen to use the New York data from the exercises (https://cocl.us/new_york_dataset).</p>
<p>I will also be using my personal developer Foursquare to locate other Pizza restaurants.</p> 
<h3>How the data will be used</h3>
<p>The main components of our analysis and predication system, this will comprise of four stages:
    
1. Collect Data
2. Explore and Understand Data
3. Data preparation and preprocessing 
4. Modeling
</p>

<h2>Collect Data</h2> 

In [3]:
## Install packages
!pip install bs4
!pip install lxml
!conda install -c conda-forge geopy --yes

     |████████████████████████████████| 102kB 10.2MB/s ta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
     |████████████████████████████████| 5.8MB 4.6MB/s eta 0:00:01�████████████████████           | 3.8MB 4.6MB/s eta 0:00:01     |████████████████████████████▏   | 5.1MB 4.6MB/s eta 0:00:01
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0  

In [4]:
## Import libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [5]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [6]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)


In [7]:
neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

## load data into the dataframe
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
neighborhoods

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


In [8]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [9]:
# Creating new Dataframe manhattan_data
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [10]:
manhattan_data.shape[0]

40

In [11]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'CLV4Q1JFT3JFZNHXENXHTTAHDZW0XIXOS5FRN3Z3X3NHFGPC' # your Foursquare ID
CLIENT_SECRET = 'QL0ZDQ24UUK4ZS4OTWZKYNNNAT0QXNRS4WKVA3I3WUNAP0VR' # your Foursquare Secret
VERSION = '20180604'
limit = 20
print('Your credentails:')
print('CLIENT_ID:'+ CLIENT_ID)
print('CLIENT_SECRET:'+ CLIENT_SECRET)

Your credentails:
CLIENT_ID:CLV4Q1JFT3JFZNHXENXHTTAHDZW0XIXOS5FRN3Z3X3NHFGPC
CLIENT_SECRET:QL0ZDQ24UUK4ZS4OTWZKYNNNAT0QXNRS4WKVA3I3WUNAP0VR


In [12]:
# Let's get the geographical coordinates of Manhattan.
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [13]:
# let's visualizat Manhattan the neighborhoods in it.
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [17]:
# Let's create a function to repeat the process to all the neighborhoods in New York
def getNearbyVenues(names, latitudes,longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names,latitudes,longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
# Write the code to run the above function on each neighborhood and create a new dataframe called venues.
venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude'],
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [19]:
print(venues.shape)
venues.head()

(4813, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


In [22]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,20,20,20,20,20,20
Annadale,12,12,12,12,12,12
Arden Heights,4,4,4,4,4,4
Arlington,6,6,6,6,6,6
Arrochar,18,18,18,18,18,18
Arverne,18,18,18,18,18,18
Astoria,20,20,20,20,20,20
Astoria Heights,11,11,11,11,11,11
Auburndale,18,18,18,18,18,18


In [23]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 349 uniques categories.


<h2>Analyse Data</h2> 

In [24]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwic

In [26]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
grouped = onehot.groupby('Neighborhood').mean().reset_index()

In [27]:
# Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Allerton----
               venue  freq
0        Pizza Place  0.15
1           Pharmacy  0.10
2        Supermarket  0.10
3        Bus Station  0.05
4  Martial Arts Dojo  0.05


----Annadale----
                 venue  freq
0  American Restaurant  0.17
1           Sports Bar  0.08
2           Restaurant  0.08
3                 Food  0.08
4        Train Station  0.08


----Arden Heights----
           venue  freq
0    Pizza Place  0.25
1  Deli / Bodega  0.25
2       Pharmacy  0.25
3    Coffee Shop  0.25
4   Optical Shop  0.00


----Arlington----
                 venue  freq
0        Deli / Bodega  0.17
1  American Restaurant  0.17
2          Coffee Shop  0.17
3             Bus Stop  0.17
4        Boat or Ferry  0.17


----Arrochar----
                venue  freq
0            Bus Stop  0.17
1       Deli / Bodega  0.11
2  Italian Restaurant  0.11
3         Pizza Place  0.06
4               Hotel  0.06


----Arverne----
            venue  freq
0       Surf Spot  0.22
1   Metro Station  

In [28]:
# Let's put that into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Pharmacy,Supermarket,Breakfast Spot,Fast Food Restaurant,Electronics Store,Chinese Restaurant,Gas Station,Bus Station,Donut Shop
1,Annadale,American Restaurant,Pub,Train Station,Pizza Place,Diner,Pharmacy,Restaurant,Park,Food,Bakery
2,Arden Heights,Pharmacy,Deli / Bodega,Pizza Place,Coffee Shop,Women's Store,Event Service,Event Space,Eye Doctor,Factory,Falafel Restaurant
3,Arlington,Intersection,Deli / Bodega,American Restaurant,Boat or Ferry,Coffee Shop,Bus Stop,Women's Store,Filipino Restaurant,Eye Doctor,Factory
4,Arrochar,Bus Stop,Deli / Bodega,Italian Restaurant,Athletics & Sports,Middle Eastern Restaurant,Supermarket,Mediterranean Restaurant,Sandwich Place,Food Truck,Bagel Shop
5,Arverne,Surf Spot,Sandwich Place,Metro Station,Beach,Wine Shop,Playground,Thai Restaurant,Pizza Place,Bed & Breakfast,Coffee Shop
6,Astoria,Seafood Restaurant,Gourmet Shop,Gym,Pub,Middle Eastern Restaurant,Greek Restaurant,Food Truck,Bakery,Bagel Shop,Falafel Restaurant
7,Astoria Heights,Supermarket,Pizza Place,Bakery,Music Venue,Bus Station,Bowling Alley,Italian Restaurant,Burger Joint,Hostel,Playground
8,Auburndale,Deli / Bodega,Noodle House,Fast Food Restaurant,Supermarket,Toy / Game Store,Gymnastics Gym,Furniture / Home Store,Athletics & Sports,Pet Store,Pharmacy
9,Bath Beach,Bubble Tea Shop,Italian Restaurant,Rental Car Location,Turkish Restaurant,Cantonese Restaurant,Sushi Restaurant,Surf Spot,Coffee Shop,Bakery,Restaurant


<h2>Clustering Manhattan</h2> 

In [57]:
# Run k-means to cluster the neighborhood into 5 clusters.
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 0, 0, 0, 2, 1, 1, 1, 1], dtype=int32)

In [60]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
manhattan_merged = manhattan_data

# merge grouped with manhattan_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head(30) 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,2,Discount Store,Coffee Shop,Yoga Studio,Bank,Gym,Donut Shop,Diner,Pharmacy,Department Store,Pizza Place
1,Manhattan,Chinatown,40.715618,-73.994279,2,Chinese Restaurant,Noodle House,Sandwich Place,Ice Cream Shop,Cocktail Bar,Museum,Bike Shop,Garden Center,Tea Room,English Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,1,Park,Wine Shop,Café,Deli / Bodega,Breakfast Spot,Frozen Yogurt Shop,Market,Tapas Restaurant,Liquor Store,Coffee Shop
3,Manhattan,Inwood,40.867684,-73.921210,1,Café,Wine Bar,Park,Yoga Studio,Pet Store,Frozen Yogurt Shop,Mexican Restaurant,Bistro,Farmers Market,Latin American Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1,Yoga Studio,Caribbean Restaurant,Mexican Restaurant,Cocktail Bar,Bakery,Pub,Wine Bar,Italian Restaurant,Mediterranean Restaurant,Japanese Restaurant
5,Manhattan,Manhattanville,40.816934,-73.957385,1,Italian Restaurant,Other Nightlife,Japanese Curry Restaurant,Beer Garden,Bike Trail,Climbing Gym,Sushi Restaurant,Supermarket,Seafood Restaurant,Park
6,Manhattan,Central Harlem,40.815976,-73.943211,1,African Restaurant,American Restaurant,Dessert Shop,Juice Bar,Spa,French Restaurant,Music Venue,Café,Caribbean Restaurant,Beer Bar
7,Manhattan,East Harlem,40.792249,-73.944182,1,Mexican Restaurant,Park,Steakhouse,Sandwich Place,Thai Restaurant,Café,Latin American Restaurant,Gym,Bakery,Clothing Store
8,Manhattan,Upper East Side,40.775639,-73.960508,1,Hotel,Chocolate Shop,Bar,Bakery,Park,Bookstore,Burrito Place,Coffee Shop,Jazz Club,Art Gallery
9,Manhattan,Yorkville,40.775930,-73.947118,1,Wine Shop,Deli / Bodega,Coffee Shop,Gym,Hobby Shop,Sandwich Place,Sushi Restaurant,Park,Bagel Shop,Asian Restaurant


In [66]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [64]:
#Commercial Places
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Park,Wine Shop,Café,Deli / Bodega,Breakfast Spot,Frozen Yogurt Shop,Market,Tapas Restaurant,Liquor Store,Coffee Shop
3,Inwood,Café,Wine Bar,Park,Yoga Studio,Pet Store,Frozen Yogurt Shop,Mexican Restaurant,Bistro,Farmers Market,Latin American Restaurant
4,Hamilton Heights,Yoga Studio,Caribbean Restaurant,Mexican Restaurant,Cocktail Bar,Bakery,Pub,Wine Bar,Italian Restaurant,Mediterranean Restaurant,Japanese Restaurant
5,Manhattanville,Italian Restaurant,Other Nightlife,Japanese Curry Restaurant,Beer Garden,Bike Trail,Climbing Gym,Sushi Restaurant,Supermarket,Seafood Restaurant,Park
6,Central Harlem,African Restaurant,American Restaurant,Dessert Shop,Juice Bar,Spa,French Restaurant,Music Venue,Café,Caribbean Restaurant,Beer Bar
7,East Harlem,Mexican Restaurant,Park,Steakhouse,Sandwich Place,Thai Restaurant,Café,Latin American Restaurant,Gym,Bakery,Clothing Store
8,Upper East Side,Hotel,Chocolate Shop,Bar,Bakery,Park,Bookstore,Burrito Place,Coffee Shop,Jazz Club,Art Gallery
9,Yorkville,Wine Shop,Deli / Bodega,Coffee Shop,Gym,Hobby Shop,Sandwich Place,Sushi Restaurant,Park,Bagel Shop,Asian Restaurant
10,Lenox Hill,Gym,Thai Restaurant,Women's Store,Pizza Place,Dessert Shop,College Academic Building,Salad Place,Smoke Shop,Burger Joint,French Restaurant
11,Roosevelt Island,Sandwich Place,Park,Coffee Shop,Greek Restaurant,Bus Stop,Liquor Store,Outdoors & Recreation,Gym,Café,School


In [65]:
#Tourist Areas & Hubs
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Discount Store,Coffee Shop,Yoga Studio,Bank,Gym,Donut Shop,Diner,Pharmacy,Department Store,Pizza Place
1,Chinatown,Chinese Restaurant,Noodle House,Sandwich Place,Ice Cream Shop,Cocktail Bar,Museum,Bike Shop,Garden Center,Tea Room,English Restaurant
27,Gramercy,Pizza Place,Coffee Shop,Yoga Studio,Thrift / Vintage Store,Irish Pub,Gourmet Shop,Liquor Store,Mexican Restaurant,Food Truck,Playground


<h3>Output Findings to CSV</h3>

In [71]:
## Output Manhattan
manhattan_merged.to_csv('Manhattan_Merged.csv',index=False)

In [70]:
## Output all neighbourhoods for alternatives
neighborhoods_venues_sorted.to_csv('NewYork.csv',index=False)

<h3>Results</h3>
<p>From analysing the data, we have uncovered that Gramercy has the most common venue for pizza places in a tourist neighborhoods. Lenox Hill and Clinton has the fourth most common venue for pizza places and Marble Hill is tenth in the commercial neighborhoods.
    
If Luigi would like to open his newest restaurant in Manhattan, Gramercy has the most amount of pizza places / venues at 15% of all venues. It would be stiff competition, however Luigi's pizza's are proven winners in other cities.
</p>

<h3>Observations</h3>
<p>When analysing the data from other neighborhoods in New York, pizza places are generally quite popular in general. 33 out of the 301 neighborhoods in New York had a 1st Most Common Venue, so pizza places are definitely popular, however it will be competitive.</p>